In [1]:
using DataFrames, CSV
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays

In [2]:
const GRB_ENV = Gurobi.Env(output_flag=0);


# data import

In [3]:
dispatch_data = CSV.read("zipcode_incident_data.csv", DataFrame)
dispatch_data

Row,ZIPCODE,INCIDENT_COUNT
,Int64,Int64
1,10003,4
2,10027,4
3,10019,3
4,10039,3
5,10035,3
6,10014,3
7,10001,3
8,10025,2
9,10029,2


In [4]:
wait_time = CSV.read("wait_time_matrix.csv",DataFrame,header=false) |> Matrix
wait_time

9×90 Matrix{Float64}:
  2845.0  32387.0  16390.0  32283.0    …     1.0e6     1.0e6      1.0e6
 27984.0  17284.0  28316.0   8441.0          1.0e6     1.0e6      1.0e6
 31549.0   4532.0  28819.0   3305.0       5982.0    9564.0        1.0e6
  1635.0   2760.0   1230.0   1169.0          1.0e6     1.0e6      1.0e6
 31129.0   2727.0   4952.0   9068.0        209.0       1.0e6  13847.0
  1285.0   2347.0   1264.0    706.0    …     1.0e6     1.0e6      1.0e6
  3316.0   1128.0   4032.0   1052.0       1503.0       1.0e6      1.0e6
  4789.0   1038.0   1548.0      1.0e6        1.0e6     1.0e6      1.0e6
  2440.0   1438.0   3092.0   3092.0          1.0e6     1.0e6      1.0e6

In [5]:
ambulance_capacity = CSV.read("area_capacity_data.csv", DataFrame)
ambulance_capacity

Row,Ambulance_i,INCIDENT_DISPATCH_AREA,CAPACITY
,Int64,String3,Int64
1,1,M1,19
2,2,M2,26
3,3,M3,28
4,4,M4,13
5,5,M5,16
6,6,M6,8
7,7,M7,20
8,8,M8,11
9,9,M9,16


In [6]:
zipcode_to_index = CSV.read("zipcode_to_index.csv", DataFrame)
zipcode_to_index

Row,Zipcode,Index
,Int64,Int64
1,10168,69
2,10013,12
3,10048,42
4,10023,21
5,10034,32
6,10014,13
7,10019,17
8,10031,29
9,10038,36


In [7]:
merged_data = leftjoin(zipcode_to_index, dispatch_data, on=:Zipcode => :ZIPCODE)
sorted_dispatch_data = sort(merged_data, :Index)
sorted_dispatch_data


Row,Zipcode,Index,INCIDENT_COUNT
,Int64,Int64,Int64?
1,10001,1,3
2,10002,2,2
3,10003,3,4
4,10004,4,missing
5,10005,5,missing
6,10006,6,missing
7,10007,7,missing
8,10009,8,1
9,10010,9,missing


In [ ]:
# convert the missing values to 0
sorted_dispatch_data.INCIDENT_COUNT .= coalesce.(sorted_dispatch_data.INCIDENT_COUNT, 0)
sorted_dispatch_data


Row,Zipcode,Index,INCIDENT_COUNT
,Int64,Int64,Int64
1,10001,1,3
2,10002,2,2
3,10003,3,4
4,10004,4,0
5,10005,5,0
6,10006,6,0
7,10007,7,0
8,10009,8,1
9,10010,9,0


## build the model:
3 useful dfs:

dispatch data with sorted index: sorted_dispatch_data   d_j

ambulance capacity: ambulance_capacity c_i 

wait time : wait_time  waittime(i,j)

In [10]:
capacity_vector = ambulance_capacity[!,"CAPACITY"]

9-element Vector{Int64}:
 19
 26
 28
 13
 16
  8
 20
 11
 16

In [11]:
demand_vector = sorted_dispatch_data[!,"INCIDENT_COUNT"]

90-element Vector{Int64}:
 3
 2
 4
 0
 0
 0
 0
 1
 0
 2
 0
 1
 3
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

# Min-Sum

In [12]:
num_areas = 9
num_zipcodes = 90

# Build the optimization model
model = Model(Gurobi.Optimizer)

# Decision variables: S[i, j] ∈ integer >=0
@variable(model, S[1:num_areas, 1:num_zipcodes] >=0, Int)

# Objective: Minimize total wait time
@objective(model, Min, sum(S[i, j] * wait_time[i, j] for i in 1:num_areas, j in 1:num_zipcodes))

# Constraints:
# Capacity constraints for each ambulance station
@constraint(model, [i in 1:num_areas], sum(S[i, j] for j in 1:num_zipcodes) <= capacity_vector[i])
# meet the demand 
@constraint(model, [j in 1:num_zipcodes], sum(S[i, j] for i in 1:num_areas) >= demand_vector[j])


optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-19
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 99 rows, 810 columns and 1620 nonzeros
Model fingerprint: 0x9e0d152b
Variable types: 0 continuous, 810 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 1e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 1387484.0000
Presolve removed 66 rows and 594 columns
Presolve time: 0.00s
Presolved: 33 rows, 216 columns, 432 nonzeros
Variable types: 0 continuous, 216 integer (108 binary)
Found heuristic solution: objective 257699.00000

Root relaxation: objective 7.934000e+04, 26 iterations, 0.00 seconds (0.00 work units)

    Nodes   

In [13]:
# Check the solution
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found:")
    solution = value.(S)
else
    println("No optimal solution found.")
end

Optimal solution found:


9×90 Matrix{Float64}:
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
  3.0  -0.0   4.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0   2.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0

In [18]:
solution

9×90 Matrix{Float64}:
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
  3.0  -0.0   4.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0   2.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0

In [19]:
nonzero_indices = findall(x -> x != 0, solution)  
nonzero_values = solution[nonzero_indices]   

println("nonzero indices: ", nonzero_indices)
println("nonzero values: ", nonzero_values)
println("number of S(i,j)>0:  ", length(nonzero_values))


nonzero indices: CartesianIndex{2}[CartesianIndex(4, 1), CartesianIndex(8, 2), CartesianIndex(4, 3), CartesianIndex(6, 8), CartesianIndex(4, 10), CartesianIndex(8, 12), CartesianIndex(8, 13), CartesianIndex(4, 14), CartesianIndex(8, 15), CartesianIndex(6, 16), CartesianIndex(6, 17), CartesianIndex(1, 23), CartesianIndex(3, 24), CartesianIndex(1, 25), CartesianIndex(9, 27), CartesianIndex(5, 29), CartesianIndex(3, 30), CartesianIndex(8, 32), CartesianIndex(9, 33), CartesianIndex(5, 34), CartesianIndex(4, 36), CartesianIndex(5, 37), CartesianIndex(6, 43), CartesianIndex(6, 50)]
nonzero values: [3.0, 2.0, 4.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0, 2.0, 3.0, 2.0, 1.0, 4.0, 2.0, 1.0, 1.0, 1.0, 3.0, 1.0, 1.0, 3.0, 1.0, 1.0]
number of S(i,j)>0:  24


In [20]:
sum(sorted_dispatch_data[!,"INCIDENT_COUNT"]) # 45, it is the total demand of all the incidents

45

In [21]:
sum(nonzero_values) # 45, so it is same as the above one, indicating that every demand is met

45.0

In [ ]:
# Assume `solution` is the matrix of decision variables and `wait_time` contains the penalties

# Find indices where the penalty is large (1e6) 
# (this is because some of the i,j does not have history dispatch data,
# so we assume these (i,j) pairs are too far away(we give it a value 1e6 during data processing) so that in history people just don't dispatch)
penalized_indices = [(i, j) for i in 1:size(wait_time, 1), j in 1:size(wait_time, 2) if wait_time[i, j] == 1e6]

# Check if any of these indices in `solution` are assigned a value bigger than 0
penalized_assignments = [(i, j) for (i, j) in penalized_indices if solution[i, j] > 0.5]

# Output the result
if !isempty(penalized_assignments)
    println("Solution includes assignments to penalized combinations:")
    println(penalized_assignments)
else
    println("No assignments were made to penalized combinations.")  # it means that our S_ij didn't choose the uncommon (i,j) pairs
end


No assignments were made to penalized combinations.


# Final Result
45 incidents, demands all met

24 pairs of (i,j) that are greater than 0


In [ ]:
assignment_matrix = zeros(Int, num_areas, num_zipcodes)

for i in 1:num_areas
    for j in 1:num_zipcodes
        if solution[i, j] > 0
            assigned_count = Int(solution[i, j])
        else
            assigned_count = 0
        end
        assignment_matrix[i, j] = assigned_count
    end
end


In [36]:
assignment_matrix

9×90 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 3  0  4  0  0  0  0  0  0  2  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  2  0  0  0  0  0  0  0  0  0  1  3     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

The following code is the assignment matrix.

In [ ]:
column_names = [string("Zipcode_", j) for j in 1:num_zipcodes]
row_names = [string("Area_M", i) for i in 1:num_areas]

df = DataFrame(assignment_matrix, Symbol.(column_names))
df.Area = row_names

# CSV.write("ambulance_assignments_matrix_named.csv", df)
println("Matrix saved to ambulance_assignments_matrix_named.csv.")


Matrix saved to ambulance_assignments_matrix_named.csv.


# Calculate Cost Matrixs

1. cost of a region (number of incidents * waittime)

2. cost of single incident

1. The following code is the cost of a region (waittime*assignment) matrix.

In [ ]:
cost_matrix = assignment_matrix .* wait_time
df_cost_matrix = DataFrame(cost_matrix, Symbol.(column_names))
df_cost_matrix.Area = row_names
CSV.write("MinSum_cost_matrix_sijWaitij.csv", df_cost_matrix)
println("cost Matrix saved to MinSum_cost_matrix_sijWaitij.csv.")


cost Matrix saved to cost_matrix_sijWaitij.csv.


2. The following code is the cost of single incident

In [44]:
wait_time

9×90 Matrix{Float64}:
  2845.0  32387.0  16390.0  32283.0    …     1.0e6     1.0e6      1.0e6
 27984.0  17284.0  28316.0   8441.0          1.0e6     1.0e6      1.0e6
 31549.0   4532.0  28819.0   3305.0       5982.0    9564.0        1.0e6
  1635.0   2760.0   1230.0   1169.0          1.0e6     1.0e6      1.0e6
 31129.0   2727.0   4952.0   9068.0        209.0       1.0e6  13847.0
  1285.0   2347.0   1264.0    706.0    …     1.0e6     1.0e6      1.0e6
  3316.0   1128.0   4032.0   1052.0       1503.0       1.0e6      1.0e6
  4789.0   1038.0   1548.0      1.0e6        1.0e6     1.0e6      1.0e6
  2440.0   1438.0   3092.0   3092.0          1.0e6     1.0e6      1.0e6

In [43]:
df

Row,Zipcode_1,Zipcode_2,Zipcode_3,Zipcode_4,Zipcode_5,Zipcode_6,Zipcode_7,Zipcode_8,Zipcode_9,Zipcode_10,Zipcode_11,Zipcode_12,Zipcode_13,Zipcode_14,Zipcode_15,Zipcode_16,Zipcode_17,Zipcode_18,Zipcode_19,Zipcode_20,Zipcode_21,Zipcode_22,Zipcode_23,Zipcode_24,Zipcode_25,Zipcode_26,Zipcode_27,Zipcode_28,Zipcode_29,Zipcode_30,Zipcode_31,Zipcode_32,Zipcode_33,Zipcode_34,Zipcode_35,Zipcode_36,Zipcode_37,Zipcode_38,Zipcode_39,Zipcode_40,Zipcode_41,Zipcode_42,Zipcode_43,Zipcode_44,Zipcode_45,Zipcode_46,Zipcode_47,Zipcode_48,Zipcode_49,Zipcode_50,Zipcode_51,Zipcode_52,Zipcode_53,Zipcode_54,Zipcode_55,Zipcode_56,Zipcode_57,Zipcode_58,Zipcode_59,Zipcode_60,Zipcode_61,Zipcode_62,Zipcode_63,Zipcode_64,Zipcode_65,Zipcode_66,Zipcode_67,Zipcode_68,Zipcode_69,Zipcode_70,Zipcode_71,Zipcode_72,Zipcode_73,Zipcode_74,Zipcode_75,Zipcode_76,Zipcode_77,Zipcode_78,Zipcode_79,Zipcode_80,Zipcode_81,Zipcode_82,Zipcode_83,Zipcode_84,Zipcode_85,Zipcode_86,Zipcode_87,Zipcode_88,Zipcode_89,Zipcode_90,Area
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,String
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Area_M1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Area_M2
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Area_M3
4,3,0,4,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Area_M4
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Area_M5
6,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Area_M6
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Area_M7
8,0,2,0,0,0,0,0,0,0,0,0,1,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Area_M8
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Area_M9


In [ ]:
using DataFrames
df_waittime_single_incident = DataFrame(
    [ifelse(df[i, j] != 0, wait_time[i, j], 0) for i in 1:size(wait_time, 1), j in 1:size(wait_time, 2)],
    names(df)[1:end-1]
)

df_waittime_single_incident


Row,Zipcode_1,Zipcode_2,Zipcode_3,Zipcode_4,Zipcode_5,Zipcode_6,Zipcode_7,Zipcode_8,Zipcode_9,Zipcode_10,Zipcode_11,Zipcode_12,Zipcode_13,Zipcode_14,Zipcode_15,Zipcode_16,Zipcode_17,Zipcode_18,Zipcode_19,Zipcode_20,Zipcode_21,Zipcode_22,Zipcode_23,Zipcode_24,Zipcode_25,Zipcode_26,Zipcode_27,Zipcode_28,Zipcode_29,Zipcode_30,Zipcode_31,Zipcode_32,Zipcode_33,Zipcode_34,Zipcode_35,Zipcode_36,Zipcode_37,Zipcode_38,Zipcode_39,Zipcode_40,Zipcode_41,Zipcode_42,Zipcode_43,Zipcode_44,Zipcode_45,Zipcode_46,Zipcode_47,Zipcode_48,Zipcode_49,Zipcode_50,Zipcode_51,Zipcode_52,Zipcode_53,Zipcode_54,Zipcode_55,Zipcode_56,Zipcode_57,Zipcode_58,Zipcode_59,Zipcode_60,Zipcode_61,Zipcode_62,Zipcode_63,Zipcode_64,Zipcode_65,Zipcode_66,Zipcode_67,Zipcode_68,Zipcode_69,Zipcode_70,Zipcode_71,Zipcode_72,Zipcode_73,Zipcode_74,Zipcode_75,Zipcode_76,Zipcode_77,Zipcode_78,Zipcode_79,Zipcode_80,Zipcode_81,Zipcode_82,Zipcode_83,Zipcode_84,Zipcode_85,Zipcode_86,Zipcode_87,Zipcode_88,Zipcode_89,Zipcode_90
,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real,Real
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2914.0,0,2320.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1560.0,0,0,0,0,0,3287.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1635.0,0,1230.0,0,0,0,0,0,0,1647.0,0,0,0,839.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,540.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2053.0,0,0,0,0,2141.0,0,0,1851.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,603.0,0,0,0,0,0,0,0,1235.0,909.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,478.0,0,0,0,0,0,0,362.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,1038.0,0,0,0,0,0,0,0,0,0,7968.0,820.0,0,753.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,737.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3020.0,0,0,0,0,0,2822.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
CSV.write("MinSum_cost_matrix_single_incident.csv", df_cost_matrix)


# In order to plot the distribution of waittime, we will have to 

Create a list such that:

If df(i, j) is 0, do not add anything to the list.

If df(i, j) is 1, add the corresponding wait_time(i, j) value to the list once.

If df(i, j) is 2, add the wait_time(i, j) value twice, and so on.

In [55]:
# Assuming `wait_time` is a matrix and `df` is a DataFrame

# Initialize an empty list to store the result
result_list = []

# Iterate through each cell of the DataFrame
for i in 1:size(df, 1)  # Iterate over rows
    for j in 1:size(df, 2)  # Iterate over columns
        # Get the value of df[i, j]
        count = df[i, j]
        
        # Get the corresponding value from wait_time matrix
        value = wait_time[i, j]
        
        # Add the value to the list 'count' times
        append!(result_list, fill(value, count))
    end
end

# Now `result_list` contains the desired values
result_list

LoadError: BoundsError: attempt to access 9×90 Matrix{Float64} at index [1, 91]